# Otimização do robô PUMA 560

Otmização feija por junta individualmente

In [1]:
include("../comum.jl")
include("modelo.jl")
using Evolutionary, Plots;
pyplot();

In [ ]:
Ts = 0.08     #período de leitura
tend = 2.0    #tempo final
t0 = 0.0      # tempo inicial
popul  = 4   # população
iterac = 2   #iterações
α = 10.       #parâmetro para o erro
β = 0.01      #parâmetro para o jerk
γ = 0.1       #parâmetro para o torque
per = 1/2     #inicio da leitura do vetor a parti de per do comprimento total
xr = SVector{5}([1.,1.,0.5,0.6,0.5]) #referências
kp_end = AbstractFloat[]
kv_end = AbstractFloat[];

## Otimização junta 5

In [3]:
function gerador5(n)
    n = n/2
    kp = push!(zeros(n-1),rand()*rand([10.,100.,1000.,10000]))
    kv = push!(zeros(n-1),rand()*rand([10.,100.,1000.]))
    vcat(kp,kv)
end;

In [5]:
function generateCusto(junta::Integer)
    out = function custo(gain::Vector{Float64})
        kp = SMatrix{5,5}(diagm(gain[1:5]))
        kv = SMatrix{5,5}(diagm(gain[6:end]))
        x, v, t, a, ta, j, tj, τ, t_tau = pumaRobot(kp, kv, Ts, t0, tend, xr)
        sizeVector = length(x[1])

        erro_sum = 0.
        erro = -(x[junta]-xr[junta])
        erro_sum += sum(abs.(erro[floor(Integer,sizeVector*per):end])) 

        jerk_sum = 0.
        jerk_sum += sum(abs.(j[junta]))
        
        torque_sum = 0.
        torque_sum += sum(abs.(τ[junta]))
        
        erro_sum = erro_sum * α
        jerk_sum = jerk_sum * β 
        torque_sum = torque_sum * γ

        out =  erro_sum + jerk_sum + torque_sum
        out
    end
end;

In [ ]:
N = 10
result, fitness, cnt = ga(generateCusto(5), N; initPopulation = gerador5, populationSize = popul, ɛ = 0.25, selection = roulette, crossover = intermediate(0.25), mutation = domainrange(fill(0.5,N)), iterations = iterac, showBar = true)

 Progress:|█

# Resultado

In [ ]:
t_end_new = tend
kp = SMatrix{5,5}(diagm(result[1:5]))
kv = SMatrix{5,5}(diagm(result[6:end]))

x, v, t, a, ta, j, tj, τ, t_tau = pumaRobot(kp, kv, Ts, t0, t_end_new, xr);

In [ ]:
erro = []
for i=1:7
    push!(erro, -(x[i] .- xr[i]))
end 
tabela(erro, "Erro")

In [ ]:
function plotx(i)
    p1 = plot(t,x[i], label = "desejado")
    p1= plot!([xr[i]],seriestype= :hline, label = "referência");
end
function plotx()
    p1 = plot(t,x[1], label = "simulação 1")
    p1= plot!([xr[1]],seriestype= :hline, label = "referência 1")
    p2 = plot(t,x[2], label = "simulação 2")
    p2= plot!([xr[2]],seriestype= :hline, label = "referência 2")
    p3 = plot(t,x[3], label = "simulação 3")
    p3= plot!([xr[3]],seriestype= :hline, label = "referência 3")
    p4 = plot(t,x[4], label = "simulação 4")
    p4= plot!([xr[4]],seriestype= :hline, label = "referência 4")
    p5 = plot(t,x[5], label = "simulação 5")
    p5= plot!([xr[5]],seriestype= :hline, label = "referência 5")
    plot(p1,p2,p3,p4,p5,p6,p7, layout =(5,1))
end

In [ ]:
plotx(5)

In [ ]:
tabela(j,"Jerk")

In [ ]:
tabela(τ,"Torque")